Index
1. Objective
2. Data Understanding
3. Exploratory Data Analysis (EDA)
    1. Descriptive Analysis of CustomerLifetimeValue
    2. Descriptive Analysis of Monthly Premium Auto(MPA)
    3. Descriptive Analysis ofTotalClaimAmount (TCA)
    4. Descriptive Analysis of other variables.
4. Inferential Statistics
    1. Effect of Insurance Coverage on Customer Life Time Value (CLV)
    2. Effect of Education on Customer Life Time Value (CLV)
    3. Effect of Employment Status on Customer Life Time Value (CLV)
    4. Effect of Gender on Customer Life Time Value (CLV)
    5. Effect of Location on Customer Life Time Value (CLV)
    6. Effect of Marital Status on Customer Life Time Value (CLV)
    7. Effect of Policy Type on Customer Life Time Value (CLV)
    8. Effect of Renew Offer Type on Customer Life Time Value (CLV)
    9. Effect of Sales Channel on Customer Life Time Value (CLV)
    10.Effect of Vehicle Class on Customer Life Time Value (CLV)
    11.Effect of Vehicle Size on Customer Life Time Value (CLV)
    12.Effect of States on Customer Life Time Value (CLV)
    13.Effect of Policy on Customer Life Time Value (CLV)
5. Regression Analysis with Continuous Variables
    1. Model Interpretation
    2. Residuals Analysis
6. Assumpation Testing of Linear Regression Analysis
    1. Detecting multicollinearity
    2. Detecting Homoscedasticity
    3. Detecting Autocorrelation
    4. Detecting MAPE
7. Removing Heteroskedasticity
8. Summary
9. Business Recommendation

# 1. Objective :-Analyzing the Impact of Various Factors on Customer Life-Time Value (CLV) in an Auto Insurance Company Dataset.

The lifetime value of a customer, or customer lifetime value (CLV), represents the total amount of money a
customer is expected to spend in business, or on products, during their lifetime. This is an important
figure to know because it helps company to make decisions about how much money to invest in
acquiring new customers and retaining existing ones.

Delving into the Influence of Various Factors on Customer Life-Time Value (CLV) in an Auto Insurance Company Dataset. 
This analysis aims to uncover the relationships between different variables and CLV, providing insights that 
can inform strategic decisions related to customer acquisition and retention efforts.


In [ ]:
# Importing packages
library(tidyverse)
library(car) 
library(zoo)
library(lmtest) 
library(dplyr) 
library(stringr)
library(caret)
library(ggplot2) 
library(timeDate)

In [ ]:
# Reproduce the same results always
set.seed(123) 

In [ ]:
# Reading in the data file
InsuranceData <- read.csv("/Users/maverick/Documents/BUAN 6312/AutoInsurance.csv")
view(InsuranceData)
head(InsuranceData) # Checking top 6 observations of dataset
tail(InsuranceData) # Checking bottom 6 observations of dataset

# Specify the variables of interest
variables_of_interest <- c("Customer.Lifetime.Value", "Income", "Monthly.Premium.Auto",
                            "Months.Since.Last.Claim", "Months.Since.Policy.Inception",
                            "Number.of.Open.Complaints", "Number.of.Policies", "Total.Claim.Amount")

# Calculate summary statistics
summary_stats <- summary(InsuranceData[variables_of_interest])

std_dev <- sapply(InsuranceData[variables_of_interest], sd)

# Calculate range (min, max)
range_stats <- data.frame(
  variable = variables_of_interest,
  min = sapply(InsuranceData[variables_of_interest], min),
  max = sapply(InsuranceData[variables_of_interest], max),
  sd = std_dev
)

# Display the results
print("Summary Statistics:")
print(summary_stats)

print("\nRange Statistics:")
print(range_stats)

# Calculate counts and percentages for "Response"
response_counts <- table(InsuranceData$Response)
response_percentages <- prop.table(response_counts) * 100

# Calculate counts and percentages for "Coverage"
coverage_counts <- table(InsuranceData$Coverage)
coverage_percentages <- prop.table(coverage_counts) * 100

# Calculate counts and percentages for "Education"
education_counts <- table(InsuranceData$Education)
education_percentages <- prop.table(education_counts) * 100

# Calculate counts and percentages for "Vehicle.Class"
vehicle_class_counts <- table(InsuranceData$Vehicle.Class)
vehicle_class_percentages <- prop.table(vehicle_class_counts) * 100

# Display the results
print("Response:")
print(data.frame(Response = names(response_counts), Count = as.numeric(response_counts), Percentage = response_percentages))

print("\nCoverage:")
print(data.frame(Coverage = names(coverage_counts), Count = as.numeric(coverage_counts), Percentage = coverage_percentages))

print("\nEducation:")
print(data.frame(Education = names(education_counts), Count = as.numeric(education_counts), Percentage = education_percentages))

print("\nVehicle Class:")
print(data.frame(Vehicle_Class = names(vehicle_class_counts), Count = as.numeric(vehicle_class_counts), Percentage = vehicle_class_percentages))



# 2. Data Understanding

In [ ]:
# Remove Customer ID from data set.
InsuranceData <- InsuranceData[,-c(1)] 


In [ ]:
#Cleaning the data
colnames(InsuranceData)
colnames(InsuranceData) <- str_replace_all(colnames(InsuranceData),"[.]","")
colnames(InsuranceData)


In [ ]:
# Data Understanding
dim(InsuranceData)
str(InsuranceData)


**The provided dataset has lots of details :**
1. There are 9134 Observations of 24 Variable
2. There are mix of categorical and continous DataType.
3. Dependent Variable is Customer Life Time Value as we have to predict the CLV.
4. Independent Variables are: Customer, StateCustomerLifetimeValue, Response, Coverage,
Education, EffectiveToDate, EmploymentStatus, Gender, Income, LocationCode, MaritalStatus,
MonthlyPremiumAuto, MonthsSinceLastClaim, MonthsSincePolicyInception,
NumberofOpenComplaints, NumberofPoliciesPolicyType, Policy, RenewOfferType,
SalesChannel, TotalClaimAmountVehicleClass, VehicleSize
5. Continues Independed Variables are : CustomerLifetimeValue, Income,MonthlyPremiumAuto,
MonthsSinceLastClaim, MonthsSincePolicyInception, NumberofOpenComplaints,
NumberofPolicies, TotalClaimAmount
6. There are no null values, so no further action required to replace missing or null values.
7. “Customer” column is serial number so it is insignificat for analysis and removed from the
dataset.

In [ ]:
# Checking null values in each column and storing the value in a data frame na_counts
na_counts <- sapply(InsuranceData, function(y) sum(is.na(y)))
na_counts <- data.frame(na_counts)
na_counts

There are no null values, so no further action required to replace missing or null values.

In [ ]:
# Unique Values of each column
sapply(InsuranceData, data.table::uniqueN)


# 3. Exploratory Data Analysis (EDA)

In this section we perform initial investigations on insurance data so as to discover patterns and to
check assumptions with the help of summary statistics and graphical representations.

**3.1. Descriptive Analysis of CustomerLifetimeValue**


In [ ]:
range(InsuranceData$CustomerLifetimeValue)
mean(InsuranceData$CustomerLifetimeValue)
sd(InsuranceData$CustomerLifetimeValue)
summary(InsuranceData$CustomerLifetimeValue)

var(InsuranceData$CustomerLifetimeValue)
skewness(InsuranceData$CustomerLifetimeValue)
kurtosis(InsuranceData$CustomerLifetimeValue) 

#hist(InsuranceData$CustomerLifetimeValue, col = "#FF5733", xlab = "CLV")
hist(InsuranceData$CustomerLifetimeValue, breaks = (max(InsuranceData$CustomerLifetimeValue) - min(InsuranceData$CustomerLifetimeValue))/100, freq = FALSE, main = "CLV Histogram", xlab = "CLV", border = "#FF5733")

1. Maximum CLV is $83325.381 and the minimum CLV is $1898.008.
1. Mean of CLV is $8005 and the Median is $5780.
1. The Variance in CLV is 47210196 and the Standard Deviation is 6870.968.
1. Skewness is 4.031284. CLV is positive skewed and most values are concentrated on the left of
1. the mean value, yet all the extreme values are on the right of the mean value.
1. Kurtosis is 13.81163. Since kurtosis > 3, means distribution has thicker tails than normal
1. distribution and have more outliers (extreme values).
1. This means that the distribution of CLV is positively skewed (as expected) and is heavily
1. Leptokurtic.
1. These results indicate a distribution that is heavily skewed with a very large tail.
1. There are a LOT of Customers with low CLV. Very few customers with high CLV.
1. This can be visually understood using the Histogram.

**3.2. Descriptive Analysis of Monthly Premium Auto(MPA)**

In [ ]:
range(InsuranceData$MonthlyPremiumAuto)
mean(InsuranceData$MonthlyPremiumAuto)
sd(InsuranceData$MonthlyPremiumAuto)
summary(InsuranceData$MonthlyPremiumAuto)
var(InsuranceData$MonthlyPremiumAuto)
skewness(InsuranceData$MonthlyPremiumAuto)
kurtosis(InsuranceData$MonthlyPremiumAuto)
cor(InsuranceData$MonthlyPremiumAuto,InsuranceData$CustomerLifetimeValue)

#hist(InsuranceData$MonthlyPremiumAuto, col = "#00AFBB", xlab = "Monthly Premium Auto")
hist(InsuranceData$MonthlyPremiumAuto, breaks = (max(InsuranceData$MonthlyPremiumAuto) - min(InsuranceData$MonthlyPremiumAuto))/1, freq = FALSE, main = "Monthly Premium Histogram", xlab = "Monthly Premium", border = "#00AFBB")

plot(x=InsuranceData$MonthlyPremiumAuto, y=InsuranceData$CustomerLifetimeValue, col="#00AFBB", cex=1, xlab="MonthlyPremiumAuto", ylab="CustomerLifetimeValue",
       main="Scatterplot of MPA vs CLV")


1. Maximum MPA is 298 and the minimum MPA is 61
1. Mean of MPA is 93.21929 and the Median is 84.00
1. The Variance in MPA is 1183.908 and the Standard Deviation is 34.40797
1. Skewness is 2.122849. MPA is positive skewed and most values are concentrated on the left of
1. the mean value, yet all the extreme values are on the right of the mean value.
1. Kurtosis is 6.187546. Since kurtosis > 3, means distribution has thicker tails than normal
1. distribution and have more outliers (extreme values).
1. There is a Positive Corelation of 39.62 % of MPA with CLV. From scatter plot, it is clearly
1. visible that on MPA, CLV is also Increasing.7.
1. Monthly premiums follow a trend similar to CLV although the distribution is NOT as skewed
1. or as long tailed as CLV. This can be visually seen in the Histogram.

**3.3. Descriptive Analysis ofTotalClaimAmount (TCA)**

In [ ]:
range(InsuranceData$TotalClaimAmount)
mean(InsuranceData$TotalClaimAmount)
sd(InsuranceData$TotalClaimAmount)
summary(InsuranceData$TotalClaimAmount)
var(InsuranceData$TotalClaimAmount)
skewness(InsuranceData$TotalClaimAmount)
kurtosis(InsuranceData$TotalClaimAmount) 
cor(InsuranceData$TotalClaimAmount,InsuranceData$CustomerLifetimeValue)

#hist(InsuranceData$TotalClaimAmount, col = "#FC4E07", xlab = "Total Claim Amount")
hist(InsuranceData$TotalClaimAmount, breaks = (max(InsuranceData$TotalClaimAmount) - min(InsuranceData$TotalClaimAmount))/10, freq = FALSE, main = "Total Claim Amount Histogram", xlab = "Total Claim Amount", border = "#FC4E07")

plot(x=InsuranceData$TotalClaimAmount, y=InsuranceData$CustomerLifetimeValue, col="#FC4E07", cex=1, xlab="TotalClaimAmount", ylab="CustomerLifetimeValue",
     main="Scatterplot of TCA vs CLV")


1. Minimum TCA is $0.099007 and the maximum TCA is $2893.239678
1. Mean of TCA is $434.0888 and the Median is $383.945
1. The Variance in TCA is 84390.3 and the Standard Deviation is 290.5001
1. Skewness is 1.714403. TCA is positive skewed and most values are concentrated on the left of
1. the mean value, yet all the extreme values are on the right of the mean value.
1. Kurtosis is 5.973506. Since kurtosis > 3, means TCA distribution has thicker tails than normal
1. distribution and have more outliers (extreme values).
1. There is a Positive Corelation of 22.65 % of TCA with CLV. From scatter plot, it is clearly
1. visible that on TCA, CLV is also Increasing.
1. Total Claim amounts also follow a trend similar to CLV and MPA although the distribution is
1. NOT as skewed or as long tailed as MPA. This can be visually seen in the Histogram.

**This means that variation in data is CLV > MPA > TCA**

**3.4 Descriptive Analysis of other variables:**

In [ ]:
cor(InsuranceData$Income,InsuranceData$CustomerLifetimeValue)
plot(x=InsuranceData$Income, y=InsuranceData$CustomerLifetimeValue, col="#FC4E07", cex=1, xlab="Income", ylab="CustomerLifetimeValue",main="Scatterplot of Income vs CLV")

cor(InsuranceData$MonthsSinceLastClaim,InsuranceData$CustomerLifetimeValue)
plot(x=InsuranceData$MonthsSinceLastClaim, y=InsuranceData$CustomerLifetimeValue, col="#FC4E07", cex=1, xlab="MonthsSinceLastClaim", ylab="CustomerLifetimeValue",main="Scatterplot of MonthsSinceLastClaim vs CLV")

cor(InsuranceData$MonthsSincePolicyInception,InsuranceData$CustomerLifetimeValue)
plot(x=InsuranceData$MonthsSincePolicyInception, y=InsuranceData$CustomerLifetimeValue, col="#FC4E07", cex=1, xlab="MonthsSinceLastClaim", ylab="CustomerLifetimeValue",main="Scatterplot of MonthsSincePolicyInception vs CLV")

cor(InsuranceData$NumberofOpenComplaints,InsuranceData$CustomerLifetimeValue)
plot(x=InsuranceData$NumberofOpenComplaints, y=InsuranceData$CustomerLifetimeValue, col="#FC4E07", cex=1, xlab="NumberofOpenComplaints", ylab="CustomerLifetimeValue",main="Scatterplot of NumberofOpenComplaints vs CLV")

cor(InsuranceData$NumberofPolicies,InsuranceData$CustomerLifetimeValue)
plot(x=InsuranceData$NumberofPolicies, y=InsuranceData$CustomerLifetimeValue, col="#FC4E07", cex=1, xlab="NumberofPolicies", ylab="CustomerLifetimeValue",main="Scatterplot of NumberofPolicies vs CLV")



**The positive correlation values close to zero show that that there is no strong relationship of Income, MonthsSinceLastClaim, NumberofPolicies etc with CLV.
**

* # 4. Inferential Statistics

The most obvious candidate for Dependent Variable is CLV (CustomerLifetimeValue).
This also makes sense from a Business Perspective as we want to understand what contributes to
making a high value customer (Descriptive analysis) and maybe later on predict who is going to be
high value customer (Predictive analysis)

**4.01 Effect of Insurance Coverage on Customer Life Time Value (CLV)**


In [ ]:
ggplot(InsuranceData, aes(x=Coverage, y= CustomerLifetimeValue, fill = Coverage)) + 
  geom_boxplot() + 
  labs(x="Coverage",y = "Customer Life Time Value", fill="Coverage") + 
  ggtitle("Visualization of CLV wrt Coverage")

aggData <- aggregate(x = InsuranceData$CustomerLifetimeValue, by=list(Coverage = InsuranceData$Coverage), FUN = sum)
aggData
ggplot(data = aggData, aes(x = Coverage, y = prop.table(stat(aggData$x)), fill = Coverage, label = scales::percent(prop.table(stat(aggData$x))))) +
  geom_bar(stat="identity", position = "dodge") + 
  geom_text(stat = 'identity', position = position_dodge(.9),  vjust = -0.5, size = 3) + 
  scale_y_continuous(labels = scales::percent) + 
  labs(x = 'Coverage', y = 'CLV in Percentage', fill = 'Coverage') + 
  ggtitle("CLV Distribution by Coverage")


Customers who have taken Basic Insurance for their vehicals are more valuable then Extended or
Premium Insurance Policy holders.

**4.02 Effect of Education on Customer Life Time Value (CLV)**

In [ ]:
ggplot(InsuranceData, aes(x=Education, y= CustomerLifetimeValue, fill = Education)) + 
  geom_boxplot() + 
  labs(x="Education",y = "Customer Life Time Value", fill="Education") + 
  ggtitle("Visualization of CLV wrt Education")

aggData <- aggregate(x = InsuranceData$CustomerLifetimeValue, by=list(Education = InsuranceData$Education), FUN = sum)

ggplot(data = aggData, aes(x = Education, y = prop.table(stat(aggData$x)), fill = Education, label = scales::percent(prop.table(stat(aggData$x))))) +
  geom_bar(stat="identity", position = "dodge") + 
  geom_text(stat = 'identity', position = position_dodge(.9),  vjust = -0.5, size = 3) + 
  scale_y_continuous(labels = scales::percent) + 
  labs(x = 'Education', y = 'CLV in Percentage', fill = 'Education') + 
  ggtitle("CLV Distribution by Education")



Educated customers (with a bachelors or equivalent degree) are more valuable than others.

**4.03 Effect of Employment Status on Customer Life Time Value (CLV)**

In [ ]:
ggplot(InsuranceData, aes(x=EmploymentStatus, y= CustomerLifetimeValue, fill = EmploymentStatus)) + 
  geom_boxplot() + 
  labs(x="Employment Status",y = "Customer Life Time Value", fill="Employment Status") + 
  ggtitle("Visualization of CLV wrt Employment Status")

aggData <- aggregate(x = InsuranceData$CustomerLifetimeValue, by=list(EmploymentStatus = InsuranceData$EmploymentStatus), FUN = sum)

ggplot(data = aggData, aes(x = EmploymentStatus, y = prop.table(stat(aggData$x)), fill = EmploymentStatus, label = scales::percent(prop.table(stat(aggData$x))))) +
  geom_bar(stat="identity", position = "dodge") + 
  geom_text(stat = 'identity', position = position_dodge(.9),  vjust = -0.5, size = 3) + 
  scale_y_continuous(labels = scales::percent) + 
  labs(x = 'EmploymentStatus', y = 'CLV in Percentage', fill = 'EmploymentStatus') + 
  ggtitle("CLV Distribution by EmploymentStatus")


Employed customers are more valuable than others as compared to Retired, Unemployed or Disabled
Customers.

**4.04 Effect of Gender on Customer Life Time Value (CLV)**

In [ ]:
ggplot(InsuranceData, aes(x=Gender, y= CustomerLifetimeValue, fill = Gender)) + 
  geom_boxplot() + 
  labs(x="Gender",y = "Customer Life Time Value", fill="Gender") + 
  ggtitle("Visualization of CLV wrt Gender")


aggData <- aggregate(x = InsuranceData$CustomerLifetimeValue, by=list(Gender = InsuranceData$Gender), FUN = sum)

ggplot(data = aggData, aes(x = Gender, y = prop.table(stat(aggData$x)), fill = Gender, label = scales::percent(prop.table(stat(aggData$x))))) +
  geom_bar(stat="identity", position = "dodge") + 
  geom_text(stat = 'identity', position = position_dodge(.9),  vjust = -0.5, size = 3) + 
  scale_y_continuous(labels = scales::percent) + 
  labs(x = 'Gender', y = 'CLV in Percentage', fill = 'Gender') + 
  ggtitle("CLV Distribution by Gender")


Gender has no role to play in determining the value of a customer. Both Male and Female looks
valuable.

**4.05 Effect of Location on Customer Life Time Value (CLV)**

In [ ]:
ggplot(InsuranceData, aes(x=LocationCode, y= CustomerLifetimeValue, fill = LocationCode)) + 
  geom_boxplot() + 
  labs(x="Location",y = "Customer Life Time Value", fill="Location") + 
  ggtitle("Visualization of CLV wrt Location")

aggData <- aggregate(x = InsuranceData$CustomerLifetimeValue, by=list(LocationCode = InsuranceData$LocationCode), FUN = sum)

ggplot(data = aggData, aes(x = LocationCode, y = prop.table(stat(aggData$x)), fill = LocationCode, label = scales::percent(prop.table(stat(aggData$x))))) +
  geom_bar(stat="identity", position = "dodge") + 
  geom_text(stat = 'identity', position = position_dodge(.9),  vjust = -0.5, size = 3) + 
  scale_y_continuous(labels = scales::percent) + 
  labs(x = 'LocationCode', y = 'CLV in Percentage', fill = 'LocationCode') + 
  ggtitle("CLV Distribution by LocationCode")


Rural customers are LESS valuable than Urban customers.

**4.06 Effect of Marital Status on Customer Life Time Value (CLV) **

In [ ]:
ggplot(InsuranceData, aes(x=MaritalStatus, y= CustomerLifetimeValue, fill = MaritalStatus)) + 
  geom_boxplot() + 
  labs(x="Marital Status",y = "Customer Life Time Value", fill="Marital Status") + 
  ggtitle("Visualization of CLV wrt Marital Status")

aggData <- aggregate(x = InsuranceData$CustomerLifetimeValue, by=list(MaritalStatus = InsuranceData$MaritalStatus), FUN = sum)

ggplot(data = aggData, aes(x = MaritalStatus, y = prop.table(stat(aggData$x)), fill = MaritalStatus, label = scales::percent(prop.table(stat(aggData$x))))) +
  geom_bar(stat="identity", position = "dodge") + 
  geom_text(stat = 'identity', position = position_dodge(.9),  vjust = -0.5, size = 3) + 
  scale_y_continuous(labels = scales::percent) + 
  labs(x = 'MaritalStatus', y = 'CLV in Percentage', fill = 'MaritalStatus') + 
  ggtitle("CLV Distribution by MaritalStatus")


Married customers are buying more auto insurance and adding more value to company.

**4.07 Effect of Policy Type on Customer Life Time Value (CLV)**

In [ ]:
ggplot(InsuranceData, aes(x=PolicyType, y= CustomerLifetimeValue, fill = PolicyType)) + 
  geom_boxplot() + 
  labs(x="Policy Type",y = "Customer Life Time Value", fill="Policy Type") + 
  ggtitle("Visualization of CLV wrt Policy Type")

aggData <- aggregate(x = InsuranceData$CustomerLifetimeValue, by=list(PolicyType = InsuranceData$PolicyType), FUN = sum)

ggplot(data = aggData, aes(x = PolicyType, y = prop.table(stat(aggData$x)), fill = PolicyType, label = scales::percent(prop.table(stat(aggData$x))))) +
  geom_bar(stat="identity", position = "dodge") + 
  geom_text(stat = 'identity', position = position_dodge(.9),  vjust = -0.5, size = 3) + 
  scale_y_continuous(labels = scales::percent) + 
  labs(x = 'PolicyType', y = 'CLV in Percentage', fill = 'PolicyType') + 
  ggtitle("CLV Distribution by PolicyType")


Customers having thier own Persoanl Policy are more valuable to company then Corporate and Special
Insurance policy holder.

**4.08 Effect of Renew Offer Type on Customer Life Time Value (CLV)**

In [ ]:
ggplot(InsuranceData, aes(x=RenewOfferType, y= CustomerLifetimeValue, fill = RenewOfferType)) + 
  geom_boxplot() + 
  labs(x="Renew Offer Type",y = "Customer Life Time Value", fill="Renew Offer Type") + 
  ggtitle("Visualization of CLV wrt Renew Offer Type")

aggData <- aggregate(x = InsuranceData$CustomerLifetimeValue, by=list(RenewOfferType = InsuranceData$RenewOfferType), FUN = sum)

ggplot(data = aggData, aes(x = RenewOfferType, y = prop.table(stat(aggData$x)), fill = RenewOfferType, label = scales::percent(prop.table(stat(aggData$x))))) +
  geom_bar(stat="identity", position = "dodge") + 
  geom_text(stat = 'identity', position = position_dodge(.9),  vjust = -0.5, size = 3) + 
  scale_y_continuous(labels = scales::percent) + 
  labs(x = 'RenewOfferType', y = 'CLV in Percentage', fill = 'RenewOfferType') + 
  ggtitle("CLV Distribution by RenewOfferType")


Offers 1 and Offer 2 attracts more customers.

**4.09 Effect of Sales Channel on Customer Life Time Value (CLV)**

In [ ]:
ggplot(InsuranceData, aes(x=SalesChannel, y= CustomerLifetimeValue, fill = SalesChannel)) + 
  geom_boxplot() + 
  labs(x="Sales Channel",y = "Customer Life Time Value", fill="Sales Channel") + 
  ggtitle("Visualization of CLV wrt Sales Channel")


aggData <- aggregate(x = InsuranceData$CustomerLifetimeValue, by=list(SalesChannel = InsuranceData$SalesChannel), FUN = sum)

ggplot(data = aggData, aes(x = SalesChannel, y = prop.table(stat(aggData$x)), fill = SalesChannel, label = scales::percent(prop.table(stat(aggData$x))))) +
  geom_bar(stat="identity", position = "dodge") + 
  geom_text(stat = 'identity', position = position_dodge(.9),  vjust = -0.5, size = 3) + 
  scale_y_continuous(labels = scales::percent) + 
  labs(x = 'SalesChannel', y = 'CLV in Percentage', fill = 'SalesChannel') + 
  ggtitle("CLV Distribution by SalesChannel")


Call Center is not performing well comparerd to other channels throughout the country (in terms of
high value customers)

**4.10 Effect of Vehicle Class on Customer Life Time Value (CLV)**

In [ ]:
ggplot(InsuranceData, aes(x=VehicleClass, y= CustomerLifetimeValue, fill = VehicleClass)) + 
  geom_boxplot() + 
  labs(x="Vehicle Class",y = "Customer Life Time Value", fill="Vehicle Class") + 
  ggtitle("Visualization of CLV wrt Vehicle Class")

aggData <- aggregate(x = InsuranceData$CustomerLifetimeValue, by=list(VehicleClass = InsuranceData$VehicleClass), FUN = sum)

ggplot(data = aggData, aes(x = VehicleClass, y = prop.table(stat(aggData$x)), fill = VehicleClass, label = scales::percent(prop.table(stat(aggData$x))))) +
  geom_bar(stat="identity", position = "dodge") + 
  geom_text(stat = 'identity', position = position_dodge(.9),  vjust = -0.5, size = 3) + 
  scale_y_continuous(labels = scales::percent) + 
  labs(x = 'VehicleClass', y = 'CLV in Percentage', fill = 'VehicleClass') + 
  ggtitle("CLV Distribution by VehicleClass")


Customers having Four-Door car and SUV are more valuable.

**4.11 Effect of Vehicle Size on Customer Life Time Value (CLV)**

In [ ]:
ggplot(InsuranceData, aes(x=VehicleSize, y= CustomerLifetimeValue, fill = VehicleSize)) + 
  geom_boxplot() + 
  labs(x="Vehicle Size",y = "Customer Life Time Value", fill="Vehicle Size") + 
  ggtitle("Visualization of CLV wrt Vehicle Size")

aggData <- aggregate(x = InsuranceData$CustomerLifetimeValue, by=list(VehicleSize = InsuranceData$VehicleSize), FUN = sum)

ggplot(data = aggData, aes(x = VehicleSize, y = prop.table(stat(aggData$x)), fill = VehicleSize, label = scales::percent(prop.table(stat(aggData$x))))) +
  geom_bar(stat="identity", position = "dodge") + 
  geom_text(stat = 'identity', position = position_dodge(.9),  vjust = -0.5, size = 3) + 
  scale_y_continuous(labels = scales::percent) + 
  labs(x = 'VehicleSize', y = 'CLV in Percentage', fill = 'VehicleSize') + 
  ggtitle("CLV Distribution by VehicleSize")


Customers having Mid Size vehicals are adding more value to Insurance company.

**4.12 Effect of States on Customer Life Time Value (CLV)**

In [ ]:
ggplot(InsuranceData, aes(x=State, y= CustomerLifetimeValue, fill = State)) + 
  geom_boxplot() + 
  labs(x="State",y = "Customer Life Time Value", fill="State") + 
  ggtitle("Visualization of CLV wrt State")

aggData <- aggregate(x = InsuranceData$CustomerLifetimeValue, by=list(State = InsuranceData$State), FUN = sum)

ggplot(data = aggData, aes(x = State, y = prop.table(stat(aggData$x)), fill = State, label = scales::percent(prop.table(stat(aggData$x))))) +
  geom_bar(stat="identity", position = "dodge") + 
  geom_text(stat = 'identity', position = position_dodge(.9),  vjust = -0.5, size = 3) + 
  scale_y_continuous(labels = scales::percent) + 
  labs(x = 'State', y = 'CLV in Percentage', fill = 'State') + 
  ggtitle("CLV Distribution by State")


California customers are more valuable.

**4.13 Effect of Policy on Customer Life Time Value (CLV)**

In [ ]:
ggplot(InsuranceData, aes(x=Policy, y= CustomerLifetimeValue, fill = Policy)) + 
  geom_boxplot() + 
  labs(x="Policy",y = "Customer Life Time Value", fill="State") + 
  ggtitle("Visualization of CLV wrt Policy")

aggData <- aggregate(x = InsuranceData$CustomerLifetimeValue, by=list(Policy = InsuranceData$Policy), FUN = sum)

ggplot(data = aggData, aes(x = Policy, y = prop.table(stat(aggData$x)), fill = Policy, label = scales::percent(prop.table(stat(aggData$x))))) +
  geom_bar(stat="identity", position = "dodge") + 
  geom_text(stat = 'identity', position = position_dodge(.9),  vjust = -0.5, size = 3) + 
  scale_y_continuous(labels = scales::percent) + 
  labs(x = 'Policy', y = 'CLV in Percentage', fill = 'Policy') + 
  ggtitle("CLV Distribution by Policy")


Personal L3 Policy is adding more value to company.

# 5. Regression Analysis with Continuous Variables

1. Dependent Variable CLV is continuous and we have seen that independent variables are mostly depending linearly with dependent Variable, So Linear Regression algorithm is best for this type of this Data.
1. The goal of the Linear regression is to find the best fit line that can accurately predict the output for the continuous dependent variable.
1. Removing qualitative variables because Linear Regression works best when variables are quantitative/numeric in nature. We have only 8 continuous independed variables.

In [ ]:
dataContinous <- dplyr::select_if(InsuranceData, ~!is.factor(.))
str(dataContinous)

In [ ]:
dim(dataContinous)

**Lnear Regression**

In [ ]:
#Regression
#lm is used to fit linear models. It can be used to carry out regression, single stratum analysis of variance and analysis of covariance.

# Creating Linear Regression Model using all the continues indepedent variables.
fit <- lm(dataContinous$CustomerLifetimeValue ~., data = dataContinous) 
summary(fit) 


**5.1.1 Model Interpretation:**

**Null Hypothesis** - None of the independant variables are significant for CLV.

**Alternate Hypothesis** - At least one of the independent variables are significant and can effect the CLV.

1. p-value of model is less than 0.05, so atleast one of the independent variables are significant.
1. p-value of MonthlyPremiumAuto, NumberofOpenComplaints and NumberofPolicies are less then 0.05, so rejecting the null hypothesis. So atlest one of them independed variables are significant and can effect the CLV.
1. However R squared is very low, only 16.02% of the variance found in the CLV can be explained by Income, MPA, MonthsSinceLastClaim, MonthsSincePolicyInception, NumberofOpenComplaints, NumberofPolicies, TCA.
1. Adjusted R squared is 0.1537 which is less than R squared.
1. Residual standard error is 6322 which is very very high, so it means the actual CLV will deviate from the true regression line by approximately 6322 on an average. The smaller the standard error, the less the spread and the more likely it is that any sample mean is close to the population mean. A small standard error is thus a Good Thing.
1. Gap between R-squared and Adjusted R-squared is 1.4% only, which is good. Typically the more non-significant variables you add into the model, the gap between two increases.
1. F-statistic: 6.958 - The lower the F-statistic, the closer to a non-significant model. So F-statistic is low means it is not very significant model.

**5.1.2. Rerun Model**


There are more than one insignificant variables in the model, so need to run the model again with only
significant variables.

In [ ]:
new_fit <- lm(dataContinous$CustomerLifetimeValue ~ 
              MonthlyPremiumAuto + NumberofOpenComplaints + NumberofPolicies + TotalClaimAmount, 
              data = dataContinous) 
summary(new_fit) 


The estimated regression line equation can be written as follow:


CLV = 433.96 + 84.21 MPA - 237.18 NoOC + 76.59 NoP - 0.96 TCA

**New Model Interpretation**


**Null Hypothesis** - None of the independent variables are significant for CLV.

**Alternate Hypothesis** – At least one of the independent variables are significant and can effect the
CLV.

1. p-value of MonthlyPremiumAuto, NumberofOpenComplaints, NumberofPolicies and TotalClaimAmount is less than 0.05 so they are significantly impact the CLV.
1. Coefficients of Independent Variables :-

    i. MonthlyPremiumAuto : 84.21. One unit increase in MonthlyPremiumAuto will increase CLV by 84.21

    ii. NumberofOpenComplaints : -237.18. One unit increase in NumberofOpenComplaints will decrease CLV by 237.18

    iii. NumberofPolicies : 76.59. One unit increase in NumberofPolicies will increase CLV by 76.59

    iv. TotalClaimAmount : -0.96. one unit increase in TotalClaimAmount will decrease by 0.96

1. So the customers having more number of policies with high monthly premium will add more value to company.
1. On the other hand, customer's Open Complaints and More Claim Amount will decrease the CLV.
1. R squared is 0.1597 which means 15.97% of dependent variable is explained by independ variable.
1. Adjusted R squared is 0.1593 which is less than R squared.

**Check the normality of Error/Residual Term (Linear Regression assumes that error are normally distributed.)**


**Null Hypotheses** - Errors are normally distributed. 

**Alt Hypothese** - Errors are not normally distributed.

In [ ]:
# Extract the residuals
residuals <- residuals(new_fit)

shapiro.test(residuals[0:5000])


p-value(0.00837) < 0.05, Null Hypotheses get rejected, and so the errors are not normally distributed.

In [ ]:
hist(residuals,col = "green")


**Residuals vs Fitted Plot**

In [ ]:
plot(new_fit, which=1, col=c("blue"))


# 6. Assumpation Testing of Linear Regression Analysis

**1. Detecting multicollinearity** - checking correlation between independent variables.

In our model, only those independent variable should exist which are not correlated with each other.
This is done using Correlation Matrix.

In [ ]:
cor_matrix <- cor(dataContinous[, c("MonthlyPremiumAuto", "NumberofOpenComplaints", "NumberofPolicies", "TotalClaimAmount")])

# Print the correlation matrix
print(cor_matrix)
library("corrplot")

# Create a correlation plot
corrplot(cor_matrix, method = "circle")

# Variance Inflation Factors
car::vif(new_fit)


Variance inflation factor (VIF) is a measure of the amount of multicollinearity in a set of multiple regression variables. 

If there is high correlation between two independed variables (high multicollinearity), then you will not be able to seperate out the impact of individual independed variable on depended variable.

Due to multicolinearity we can't define the complete impact of only one independed variable on the depended variable.


**2. Detecting Homoscedasticity** - variance for all observations are not the same

**Null Hypothesis** - Homoscedasticity is present in Residuals.

**Alternate hypothesis** - Heteroskedasticity is present in residuals.

This is done by Breusch-Pagan test.

In [ ]:
bptest(new_fit)


p-value < 0.05, so it rejects that errors have homoscedasticity. 
So errors terms have heteroscedasticity and does not have constant variance which is not good for model.

**3. Detecting Autocorrelation** - checking autocorrelation (Checking correlation between errors)

This is done Durbin-Watson Test
If D-W Statistic is around 2, then we have autocorrelation in model. and away from 2 means no
autocorrelation.

In [ ]:
dwt(new_fit)


Here D-W Statistic is 1.9975, so there is autocorrelation in the model.

# 7. Removing Heteroskedasticity

In [ ]:
library(broom)
tidy.g <- function(model,vc=vcov(model),conf.int=FALSE,conf.level=0.95){
  dt <- tidy(model,conf.int=conf.int,conf.level=conf.level)
  dt$std.error <- sqrt(diag(vc))
  dt$statistic <- dt$estimate/dt$std.error
  dt$p.value <- 2*pt(-abs(dt$statistic),df=glance(model)$df.residual)
  if(conf.int){
    dt$conf.low <- dt$estimate+qt((1-conf.level)/2,df=glance(model)$df.residual)*dt$std.error
    dt$conf.high <- dt$estimate-qt((1-conf.level)/2,df=glance(model)$df.residual)*dt$std.error
  }
  return(dt)
}
tidy.w <- function(model,...)tidy.g(model,vc=sandwich::vcovHC(model),...)
hdf_fit <- tidy.w(new_fit)
summary(new_fit)
hdf_fit


The new hdf_fit model has rectified the heteroskedasticity problem and improved the standard errors of the coefficients.

# 8. Summary

1. There are a lot of Customers with low CLV. Very few customers with high CLV.
1. Customers who have taken Basic Insurance for their vehicle are more valuable then Extended or Premium Insurance Policy holders.
1. Educated Employed customers (with a bachelors or equivalent degree) are more valuable than Retired, Unemployed or Disabled Customers.
1. Gender has no role to play in determining the value of a customer. Both Male and Female looks valuable.
1. Marital customers are buying more auto insurance and adding more value to company.
1. Rural customers are LESS valuable than Urban customers.
1. Customers having their own Personal Policy are more valuable to company then Corporate and Special Insurance policy holder.
1. Offers 1 and Offer 2 attracts more customers.
1. Call Center is not performing well compared to other channels throughout the country (in terms of high value customers)
1. Customers having Mid Size vehicles, Four-Door car or SUV are more valuable.
1. California customers are adding more value to the company.
1. Personal L3 Policy is adding more value to company.xiii.
1. The customers having more number of policies with high monthly premium will add more value to company. On the other hand, customer's Open Complaints and More Claim Amount will decrease the CLV.

# 9. Business Recommendation

This report represents our analysis for the XYZ Insurance company. It is our opinion that
based on the data provided, targeting appropriate customers could increase the Customer
Lifetime Value. The two proposed changes are as follows:

A) Insurance company should target educated married employed customers from
Urban areas having Mid Size vehicles to increase the Customer Lifetime Value
(CLV) increase.

B) On the other hand, if customer's Open Complaints would not be resolved soon and
claim amount would not bring down, then both could decrease the Customer
Lifetime Value (CLV).

C) About 38% value was added by the agents to the company whereas call centers
added only 20% value. So agents should be preferred over call centers while
selling the auto insurance to customers.

D) Factors which are responsible for increasing the CLV are Monthly Premium and
Number of Policies, however Open Complaints and Claim Amount can decrease
the CLV.